In [1]:
type direction = Left | Right
type state = string 

(* Komentarji v datoteki so za lazje razumevanje ob morebitnem zagovoru *)

type direction = Left | Right


type state = string


In [119]:
module Tape = struct 
  type t = {
    negative : char list;
    head : char; 
    positive : char list;  
  }
  
  let make str = 
    match (String.to_seq str |> List.of_seq) with
    | [] -> { negative = []; head = ' '; positive = [] }
    | x :: xs -> { negative = []; head = x; positive = xs }
  
  let read (tape : t) : char =
    tape.head

  let write (ch : char) (tape : t) : t =
    { tape with head = ch }

  let move (direction : direction) (tape : t) : t = 
    match direction with 
    | Left -> (
        match tape.negative with 
        | [] -> { negative = []; head = ' '; positive = tape.head :: tape.positive }
        | x :: xs -> { negative = xs; head = x; positive = tape.head :: tape.positive }
      )
    | Right -> (
        match tape.positive with 
        | [] -> { negative = tape.head :: tape.negative; head = ' '; positive = [] } 
        | x :: xs -> { negative = tape.head :: tape.negative; head = x; positive = xs }
      )

  let print (tape : t) : unit = 
    print_string (List.to_seq (List.rev tape.negative) |> String.of_seq);
    print_char tape.head;
    print_string (List.to_seq tape.positive |> String.of_seq);
    print_newline ();
    print_string (String.make (List.length tape.negative) ' ');
    print_char '^';
    print_newline ()
end




module Tape :
  sig
    type t = { negative : char list; head : char; positive : char list; }
    val make : String.t -> t
    val read : t -> char
    val write : char -> t -> t
    val move : direction -> t -> t
    val print : t -> unit
  end


In [120]:
let primer_trak_1 = Tape.(
  make "ABCDE"
  |> move Right
  |> move Right
  |> write '!'
  |> move Left
  |> print
)

AB!DE
 ^


val primer_trak_1 : unit = ()


In [121]:
let primer_trak_2 = Tape.(
  make "ABCDE"
  |> move Left
  |> move Left
  |> move Right
  |> move Right
  |> move Right
  |> move Right
  |> write '!'
  |> print
)

  AB!DE
    ^


val primer_trak_2 : unit = ()


In [142]:
module Machine : sig
  type t
  val make : state -> state list -> t
  val initial : t -> state
  val add_transition : state -> char -> state -> char -> direction -> t -> t
  val step : t -> state -> Tape.t -> (state * Tape.t) option
end = struct

  module StringMap = Map.Make(String) 

  type t = { 
    initial_state : state;
    states : int StringMap.t; 
    transitions : (state * char * direction) option array array 
  }

  let add_transition st_state st_head end_state end_head direction machine = {
    machine with 
    transitions = 
      let new_transitions = Array.map (fun row -> Array.copy row) machine.transitions in (* Create copy of starting transition array *)
      let x = StringMap.find st_state machine.states in (* Locate index of st_state in array *)
      let y = Char.code st_head in (* Assign head value a unique code *)
      new_transitions.(x).(y) <- Some (end_state, end_head, direction); (* Assign the new transition at location (state_index, head_code) *)
      new_transitions
  } (* Return new machine with only transitions modified *)

  let make initial_state states =  
    { 
      initial_state = initial_state; 
      states = List.mapi (fun idx state -> (state, idx)) (initial_state :: states)
               |> List.to_seq |> StringMap.of_seq; (* Generate int code of all state, create sequence of (state, index) pairs, create StringMap from sequence *)
      transitions = Array.make_matrix (List.length (initial_state :: states)) 128 None (* Ensure correct size and fill matrix with 'None' *)
    }

  let initial (machine : t) : state = 
    machine.initial_state 

  let step machine state tape = 
    let x = StringMap.find state machine.states in (* Find index of current state *)
    let y = Char.code (Tape.read tape) in (* Find index of current head *)
    match machine.transitions.(x).(y) with
    | None -> None (* No further instructions were found in array, stop machine *)
    | Some (state', head', direction) -> (* New instructions found, change state, head and move tape *)
      let new_tape = tape |> Tape.write head' |> Tape.move direction in
      Some (state', new_tape)
end


module Machine :
  sig
    type t
    val make : state -> state list -> t
    val initial : t -> state
    val add_transition :
      state -> char -> state -> char -> direction -> t -> t
    val step : t -> state -> Tape.t -> (state * Tape.t) option
  end


In [144]:
let binary_increment =
  Machine.(
    make "right" [ "carry"; "done" ]
    |> add_transition "right" '1' "right" '1' Right
    |> add_transition "right" '0' "right" '0' Right
    |> add_transition "right" ' ' "carry" ' ' Left
    |> add_transition "carry" '1' "carry" '0' Left
    |> add_transition "carry" '0' "done" '1' Left
    |> add_transition "carry" ' ' "done" '1' Left
  )

val binary_increment : Machine.t = <abstr>


In [201]:
let slow_run (machine : Machine.t) (starting_string) : unit =
  let rec run state tape =
    print_endline "Tape: ";
    Tape.print tape;
    print_endline ("State: " ^ state); (* Display tape, head position and current state of machine *)
    match Machine.step machine state tape with
      | None -> () (* If no further instructions, end *)
      | Some (state', tape') -> (* Continue running *)
        run state' tape'
  in
  run (Machine.initial machine) (Tape.make starting_string) (* Create new machine with starting tape and display each step*)


val slow_run : Machine.t -> String.t -> unit = <fun>


In [202]:
let primer_slow_run = slow_run binary_increment "1011"

Tape: 
1011
^
State: right
Tape: 
1011
 ^
State: right
Tape: 
1011
  ^
State: right
Tape: 
1011
   ^
State: right
Tape: 
1011 
    ^
State: right
Tape: 
1011 
   ^
State: carry
Tape: 
1010 
  ^
State: carry
Tape: 
1000 
 ^
State: carry
Tape: 
1100 
^
State: done


val primer_slow_run : unit = ()


In [203]:
let speed_run (machine : Machine.t) (starting_string) : unit = 
  let rec run state tape =
    match Machine.step machine state tape with
      | None -> tape (* No further instructions, stop running *)
      | Some (state', tape') -> (* Continue running *)
        run state' tape'
  in 
  let stop_machine = run (Machine.initial machine) (Tape.make starting_string) in (* Finished version of machine to be printed *)
  Tape.print stop_machine

val speed_run : Machine.t -> String.t -> unit = <fun>


In [204]:
let primer_speed_run = speed_run binary_increment "1011"

1100 
^


val primer_speed_run : unit = ()


In [149]:
(* Applies list of transitions to machine *)
let for_state (state : state) (transitions : (state -> Machine.t -> Machine.t) list list) (machine: Machine.t) : Machine.t = 
  transitions
  |> List.flatten
  |> List.map (fun transition -> transition state) (* Bind transitions to the specified state *)
  |> List.fold_left (fun machine' transition -> transition machine') machine (* Apply each transition to the machine *)

(* Create list of transitions for multiple characters *)
let for_characters (heads : string) (transition : char -> state -> Machine.t -> Machine.t) : (state -> Machine.t -> Machine.t) list = (* Create list of transitions for multiple characters *)
  heads
  |> String.to_seq
  |> List.of_seq
  |> List.map transition (* Create transition for each character *)

(* Create transition for character in head *)
let for_character (head : char) (transition : char -> state -> Machine.t -> Machine.t) : (state -> Machine.t -> Machine.t) list = 
  [transition head]
  
(* Only perform move without changing state or head value *)
let move (direction : direction) : (char -> state -> Machine.t -> Machine.t) = 
  fun head state machine -> Machine.add_transition state head state head direction machine

(* Create transition to switch state, keep head and move in direction *)
let switch_and_move (state' : state) (direction : direction) : (char -> state -> Machine.t -> Machine.t) = (* *)
  fun head state machine -> Machine.add_transition state head state' head direction machine

(* Create transition to write char to head, keep same state and move *)
let write_and_move (head' : char) (direction : direction) : (char -> state -> Machine.t -> Machine.t) =
  fun head state machine -> Machine.add_transition state head state head' direction machine

(* Create transition to write char, switch state, move tape *)
let write_switch_and_move (head' : char) (state' : state) (direction : direction) : (char -> state -> Machine.t -> Machine.t) =
  fun head state machine -> Machine.add_transition state head state' head' direction machine

val for_state :
  state ->
  (state -> Machine.t -> Machine.t) list list -> Machine.t -> Machine.t =
  <fun>


val for_characters :
  string ->
  (char -> state -> Machine.t -> Machine.t) ->
  (state -> Machine.t -> Machine.t) list = <fun>


val for_character :
  char ->
  (char -> state -> Machine.t -> Machine.t) ->
  (state -> Machine.t -> Machine.t) list = <fun>


val move : direction -> char -> state -> Machine.t -> Machine.t = <fun>


val switch_and_move :
  state -> direction -> char -> state -> Machine.t -> Machine.t = <fun>


val write_and_move :
  char -> direction -> char -> state -> Machine.t -> Machine.t = <fun>


val write_switch_and_move :
  char -> state -> direction -> char -> state -> Machine.t -> Machine.t =
  <fun>


In [259]:
let binary_increment' =
  Machine.make "right" [ "carry"; "done" ]
  |> for_state "right" [
    for_characters "01" @@ move Right;
    for_character ' ' @@ switch_and_move "carry" Left
  ]
  |> for_state "carry" [
    for_character '1' @@ write_switch_and_move '0' "carry" Left;
    for_characters "0 " @@ write_switch_and_move '1' "done" Left
  ]

val binary_increment' : Machine.t = <abstr>


In [260]:
let primer_krajsi_zapis = speed_run binary_increment' "1011"

1100 
^


val primer_krajsi_zapis : unit = ()


In [281]:
(*
    - "start": mark end with X
    - "move-back": moves the tape head back to the beginning 
    - "find-char": finds the next character
    - "carry-zero": Handles the reversal logic for a '0'
    - "carry-one": Handles the reversal logic for a '1'
    - "write-zero": 
    - "write-one": 
    - "clear-x": clears temporary markers
    - "continue-write": continues writing 
    - "return-start": returns tape head to star
    - "finished": 
*)
let reverse =
  Machine.make "start" [ "move-back"; "find-char"; "carry-zero"; "carry-one"; "write-zero"; "write-one"; "clear-x"; "continue-write"; "return-start"; "finished" ]
  |> for_state "start" [
    for_characters "01" (move Right);
    for_character ' ' (write_switch_and_move 'X' "move-back" Left)
  ]
  |> for_state "move-back" [
    for_characters "01" (move Left);
    for_character ' ' (write_switch_and_move 'Y' "find-char" Right)
  ]
  |> for_state "find-char" [
    for_character ' ' (move Right);
    for_character '0' (write_switch_and_move ' ' "carry-zero" Left);
    for_character '1' (write_switch_and_move ' ' "carry-one" Left);
    for_character 'X' (write_switch_and_move ' ' "clear-x" Left)
  ]
  |> for_state "carry-zero" [
    for_character ' ' (move Left);
    for_character 'Y' (switch_and_move "write-zero" Left)
  ]
  |> for_state "carry-one" [
    for_character ' ' (move Left);
    for_character 'Y' (switch_and_move "write-one" Left)
  ]
  |> for_state "write-zero" [
    for_characters "01" (move Left);
    for_character ' ' (write_switch_and_move '0' "continue-write" Right)
  ]
  |> for_state "write-one" [
    for_characters "01" (move Left);
    for_character ' ' (write_switch_and_move '1' "continue-write" Right)
  ]
  |> for_state "clear-x" [
    for_character ' ' (move Left);
    for_character 'Y' (write_switch_and_move ' ' "return-start" Left)
  ]
  |> for_state "continue-write" [
    for_characters "01" (move Right);
    for_character 'Y' (switch_and_move "find-char" Right)
  ]
  |> for_state "return-start" [
    for_characters "01" (move Left);
    for_character ' ' (switch_and_move "finished" Right)
  ]


val reverse : Machine.t = <abstr>


In [282]:
let primer_reverse = speed_run reverse "0000111001"

 1001110000            
 ^


val primer_reverse : unit = ()


In [270]:
(*
  State Descriptions:
    - "start": Finds end of string, marks with 'M'
    - "find-bit": locates each bit from right to left for duplication
    - "read-bit": reads and erases the current bit, then directs to write its duplicate
    - "write-first-zero" / "write-first-one": writes the first duplicate bit 
    - "write-second-zero" / "write-second-one": writes the second duplicate bit and returns to find the next bit
    - "completed": /
*)
let duplicate =
  Machine.make "start" [ "find-bit"; "read-bit"; "write-first-zero"; "write-first-one"; "write-second-zero"; "write-second-one"; "completed" ]
  |> for_state "start" [
    for_characters "01" (move Right);
    for_character ' ' (write_switch_and_move 'M' "find-bit" Left)
  ]
  |> for_state "find-bit" [
    for_characters "01M" (move Left);
    for_character ' ' (switch_and_move "read-bit" Right)
  ]
  |> for_state "read-bit" [
    for_character '0' (write_switch_and_move ' ' "write-first-zero" Right);
    for_character '1' (write_switch_and_move ' ' "write-first-one" Right);
    for_character 'M' (write_switch_and_move ' ' "completed" Right)
  ]
  |> for_state "write-first-zero" [
    for_characters "01M" (move Right);
    for_character ' ' (write_switch_and_move '0' "write-second-zero" Right)
  ]
  |> for_state "write-first-one" [
    for_characters "01M" (move Right);
    for_character ' ' (write_switch_and_move '1' "write-second-one" Right)
  ]
  |> for_state "write-second-zero" [
    for_character ' ' (write_switch_and_move '0' "find-bit" Left)
  ]
  |> for_state "write-second-one" [
    for_character ' ' (write_switch_and_move '1' "find-bit" Left)
  ]


val duplicate : Machine.t = <abstr>


In [278]:
let primer_duplicate = speed_run duplicate "010011"

        001100001111
        ^


val primer_duplicate : unit = ()


In [272]:
(*
    - "move-right": moves right till empty space
    - "subtract-one": performs the subtraction of one from the binary number
    - "transfer-left": moves the tape head left
    - "write-unary": writes a unary '1' 
    - "return-to-start": returns the tape head to the starting position
    - "clear-bits": cleans up remaining bits 
    - "finished": /
*)
let to_unary =
  Machine.make "move-right" [ "subtract-one"; "transfer-left"; "write-unary"; "clear-bits"; "return-to-start"; "finished" ]
  |> for_state "move-right" [
    for_characters "01" (move Right);
    for_character ' ' (switch_and_move "subtract-one" Left)
  ]
  |> for_state "subtract-one" [
    for_character '1' (write_switch_and_move '0' "transfer-left" Right);
    for_character '0' (write_switch_and_move '1' "subtract-one" Left);
    for_character ' ' (switch_and_move "clear-bits" Right)
  ]
  |> for_state "transfer-left" [
    for_characters "01" (move Right);
    for_character ' ' (switch_and_move "write-unary" Right)
  ]
  |> for_state "write-unary" [
    for_character '1' (move Right);
    for_character ' ' (write_switch_and_move '1' "return-to-start" Left)
  ]
  |> for_state "clear-bits" [
    for_character '1' (write_and_move ' ' Right);
    for_character ' ' (switch_and_move "finished" Right)
  ]
  |> for_state "return-to-start" [
    for_character '1' (move Left);
    for_character ' ' (switch_and_move "subtract-one" Left)
  ]


val to_unary : Machine.t = <abstr>


In [273]:
let primer_to_unary = speed_run to_unary "1010"

      1111111111
      ^


val primer_to_unary : unit = ()


In [274]:
(*
    - "carry-after": handles the carry-over logic 
    - "start-marker": markes end of string
    - "return": moves the tape head back
    - "lookup": reads and processes each bit
    - "carry-before": prepares to handle carry-over 
    - "finish": clearing unnecessary bits
    - "delete_M": removes M
*)
let to_binary =
  Machine.make "start-marker" ["lookup"; "carry-before"; "carry-after"; "finish"; "return"; "delete_M"] 
  |> for_state "carry-after"[
    for_characters "0 " (write_switch_and_move '1' "return" Right);
    for_character '1' (write_and_move '0' Left)
  ]
  |> for_state "start-marker"[
    for_character '1' (move Left) ;
    for_character ' ' (write_switch_and_move 'M' "lookup" Right)
  ]
  |> for_state "return"[
    for_characters "01" (move Right) ;
    for_character 'M' (switch_and_move "lookup" Right)
  ]
  |> for_state "lookup"[
    for_character '1' (write_switch_and_move '0' "carry-before" Left) ;
    for_character '0' (move Right) ;
    for_character ' ' (switch_and_move "finish" Left)
  ]
  |> for_state "carry-before"[
    for_character '0' (move Left) ;
    for_character 'M' (switch_and_move "carry-after" Left)
  ]
  |> for_state "finish"[
    for_character '0' (write_and_move ' ' Left) ;
    for_character 'M' (write_switch_and_move ' ' "delete_M" Left)
  ]


val to_binary : Machine.t = <abstr>


In [275]:
let primer_to_binary = speed_run to_binary (String.make 42 '1')

101010                                            
     ^


val primer_to_binary : unit = ()
